<a href="https://colab.research.google.com/github/shaheryar3/Blog/blob/main/LangChainPlayGroung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [5]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [6]:
import os
print(os.getcwd())

/content


In [25]:
# prompt: install csvloader

!pip install kagglehub
!pip install --upgrade langchain-community pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 51.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [32]:
from langchain_community.document_loaders import CSVLoader
import kagglehub
import os
import pandas as pd

# Download latest version
dataset_path = kagglehub.dataset_download("utkarshx27/movies-dataset")

# Find the CSV file within the downloaded dataset directory
for filename in os.listdir(dataset_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(dataset_path, filename)
        break  # Stop after finding the first CSV file
df = pd.read_csv(file_path, encoding="utf-8", nrows=1) # Read only first row for column names
column_names = df.columns.tolist() # Get the column names as a list
print("Column names:", column_names)

# Load the dataframe, selecting only the desired columns
df = pd.read_csv(file_path, encoding="utf-8", usecols=['genres', 'overview'])

# Create a list of documents from the dataframe
docs = []
for index, row in df.iterrows():
    # Check if 'overview' is a valid string before creating a Document
    if isinstance(row['overview'], str):
        metadata = {'genres': row['genres']}  # Include genre in metadata
        docs.append(Document(page_content=row['overview'], metadata=metadata))

print(len(docs))

Column names: ['index', 'budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew', 'director']
4800


In [33]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.

{'genres': 'Action Adventure Fantasy Science Fiction'}


In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

4799

In [35]:
pip install -qU langchain-google-vertexai

In [36]:
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai
import google.auth
from google.colab import auth

auth.authenticate_user()

# Initialize the Google Cloud Project
project_id = ""
vertexai.init(project=project_id)

# Verify the credentials
credentials, project = google.auth.default()
print(f"Using project: {project}")
print(f"Using credentials: {credentials}")

# Initialize the VertexAIEmbeddings
embeddings = VertexAIEmbeddings(model="text-embedding-004")

Using project: 
Using credentials: <google.auth.compute_engine.credentials.Credentials object at 0x7bc2eeebbb10>


In [37]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.035300664603710175, -0.028474612161517143, -0.013477988541126251, 0.07369549572467804, 0.032908856868743896, 0.008364556357264519, 0.057256780564785004, -0.026831435039639473, 0.01825542002916336, 0.08729108422994614]


In [38]:
pip install -qU langchain-core

In [39]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [40]:
ids = vector_store.add_documents(documents=all_splits)

In [43]:
results = vector_store.similarity_search(
     "a  woman who meets a disillusioned man, on a train journey, and their unexpected journey together, marked by a road trip and the development of a unique bond"
)

print(results[0])

page_content='A lighthearted story about a man and a woman who seem destined to be together... and the hilarious chain of accidents that seem determined to keep them apart!' metadata={'genres': 'Comedy Drama Romance', 'start_index': 0}
